# Autosklearn API Examples

Quick, self contained snippets showing common autosklearn usage patterns.


## 1. Basic classification (train/test split)

- **Aim**: run autosklearn classification end to end on a standard dataset
- **Premise**: accuracy is sufficient for a quick demo
- **Inference**: this is the minimal pattern: split → fit → score


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from autosklearn.classification import AutoSklearnClassifier

# Built in dataset for a fast API demo
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Small search budget for demonstration
clf = AutoSklearnClassifier(time_left_for_this_task=120, per_run_time_limit=30, seed=0)
clf.fit(X_train, y_train)
print("Test accuracy", clf.score(X_test, y_test))


Test accuracy 0.9649122807017544


## 2. Basic regression (cross validation)

- **Aim**: show `AutoSklearnRegressor` and evaluation with cross validation
- **Premise**: R² is a standard regression demo metric
- **Inference**: this is the minimal pattern: fit → `cross_val_score`


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score
from autosklearn.regression import AutoSklearnRegressor

# Builtin dataset for a fast regression API demo.
X, y = fetch_california_housing(return_X_y=True)

# Small search budget for demonstration.
reg = AutoSklearnRegressor(time_left_for_this_task=180, per_run_time_limit=30, seed=1)
reg.fit(X, y)

scores = cross_val_score(reg, X, y, cv=3, scoring="r2")
print("CV R^2", scores.mean())


[WARNING] [2025-12-15 07:13:52,310:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-12-15 07:14:23,388:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-12-15 07:14:54,464:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-12-15 07:14:55,222:Client-EnsembleBuilder] No runs were available to build an ensemble from
CV R^2 0.7072033490240446


## 3. Custom metric (F1 macro)

- **Aim**: show how to plug a custom metric into autosklearn
- **Premise**: `f1_macro` weights all classes equally
- **Inference**: the chosen metric strongly influences which models/configurations win


In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import f1_macro

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Metric passed to auto sklearn guides the search objective
clf = AutoSklearnClassifier(
    time_left_for_this_task=150,
    per_run_time_limit=30,
    metric=f1_macro,
    seed=42,
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("F1_macro", f1_score(y_test, y_pred, average="macro"))


F1_macro 0.9845038974048764


## 4. Restrict estimators and preprocessors

- **Aim**: constrain the AutoML search space (faster + more controlled)
- **Inference**: restricting search is useful when you know which model families are acceptable


In [ ]:

from sklearn.datasets import load_wine
from autosklearn.classification import AutoSklearnClassifier

X, y = load_wine(return_X_y=True)

clf = AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    include={
        "classifier": ["random_forest", "gradient_boosting"],
        "feature_preprocessor": ["no_preprocessing"],
    },
    seed=3,
)

clf.fit(X, y)
print(clf.sprint_statistics())


[WARNING] [2025-12-15 07:44:02,978:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 315 not found
[WARNING] [2025-12-15 07:44:02,978:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 282 not found
[WARNING] [2025-12-15 07:44:02,978:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 521 not found
[WARNING] [2025-12-15 07:44:02,978:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 663 not found
[WARNING] [2025-12-15 07:44:02,979:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 160 not found
[WARNING] [2025-12-15 07:44:02,979:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 680 not found
[WARNING] [2025-12-15 07:44:02,979:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 14 not found
[WARNING] [2025-12-15 07:44:02,979:Client-AutoMLSMBO(3)::bdd5cbae-d989-11f0-95a7-0633c731bc54] Configuration 579 not found
[WARNING] [2025-1

## 5. Handling categorical features (ColumnTransformer)

- **Aim**: show how to manually encode categorical columns before calling autosklearn
- **Premise**: many estimators expect numeric matrices
- **Inference**: you typically need to encode categorical columns (e.g., one-hot) before fitting


In [ ]:

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier

# Toy example
X = pd.DataFrame({
    "city": ["delhi", "mumbai", "delhi", "pune", "pune", "mumbai"],
    "rooms": [2, 3, 2, 1, 2, 4],
    "area": [800, 1200, 850, 600, 950, 1500],
})
y = [0, 1, 0, 0, 1, 1]

cat_cols = ["city"]
num_cols = ["rooms", "area"]

preprocess = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols),
])

X_proc = preprocess.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_proc, y, test_size=0.33, random_state=0)

clf = AutoSklearnClassifier(time_left_for_this_task=60, per_run_time_limit=20, seed=0)
clf.fit(X_train, y_train)
print("Accuracy", clf.score(X_test, y_test))


[WARNING] [2025-12-15 07:46:15,372:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-12-15 07:46:16,266:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-12-15 07:46:17,089:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-12-15 07:46:17,965:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-12-15 07:46:18,896:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-12-15 07:46:20,336:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-12-15 07:46:21,238:Client-EnsembleBuilder] No 

## 6. Save and load a fitted model

- **Aim**: persist a trained autosklearn model artifact
- **Inference**: use this pattern to ship a trained model without retraining


In [ ]:

import joblib
from sklearn.datasets import load_iris
from autosklearn.classification import AutoSklearnClassifier

X, y = load_iris(return_X_y=True)

clf = AutoSklearnClassifier(time_left_for_this_task=60, per_run_time_limit=15, seed=5)
clf.fit(X, y)

joblib.dump(clf, "autosklearn_iris.joblib")
loaded = joblib.load("autosklearn_iris.joblib")
print("Reloaded accuracy", loaded.score(X, y))


Reloaded accuracy 0.9933333333333333


## 7. Inspect ensemble leaderboard

- **Aim**: view the top performing models and their scores
- **Inference**: leaderboard + ensemble composition helps explain what AutoML selected


In [ ]:

from sklearn.datasets import load_breast_cancer
from autosklearn.classification import AutoSklearnClassifier

X, y = load_breast_cancer(return_X_y=True)

clf = AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    ensemble_kwargs={"ensemble_size": 25},
    seed=7,
)
clf.fit(X, y)
print(clf.leaderboard())


          rank  ensemble_weight               type      cost  duration
model_id                                                              
53           1             0.44                sgd  0.015957  0.960393
32           2             0.04                mlp  0.037234  1.992943
13           3             0.04        extra_trees  0.047872  1.690032
15           4             0.04        extra_trees  0.047872  2.829963
42           5             0.04        extra_trees  0.047872  1.698408
40           6             0.04                qda  0.053191  0.860871
47           7             0.08      random_forest  0.053191  1.440655
29           8             0.08      liblinear_svc  0.058511  0.766672
28           9             0.04                lda  0.079787  0.847620
48          10             0.12  gradient_boosting  0.079787  0.834188
31          11             0.04       bernoulli_nb  0.601064  0.861253
